# ResNet

paper: https://arxiv.org/pdf/1512.03385.pdf

Implementation of ResNet50
<ul>
    <li> BN after each convolution and before activation </li>
    <li> Downsampling is performed by conv3_1, conv4_1, conv5_1 with a stride of 2 </li> 
    <li> Skipped connections: two types: identity connection and projection connection </li>
</ul>

In [2]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, MaxPool2D, GlobalAvgPool2D, Dense

In [10]:
def conv_batchnorm_relu(x, filters, kernel_size, strides=1):
    x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

In [21]:
# Identity Block

def identity_block(tensor, filters):
    # Left stream
    x = conv_batchnorm_relu(tensor, filters=filters, kernel_size=1, strides=1)
    x = conv_batchnorm_relu(x, filters=filters, kernel_size=3, strides=1)
    x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)
    x = BatchNormalization()(x)
    # Add to right stream
    x = Add()([tensor, x])
    x = ReLU()(x)
    return x

# Projection Block 

def projection_block(tensor, filters, strides):
    # Left stream
    x = conv_batchnorm_relu(tensor, filters=filters, kernel_size=1, strides=strides)
    x = conv_batchnorm_relu(x, filters=filters, kernel_size=3, strides=1)
    x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)
    x = BatchNormalization()(x)
    
    # Right Stream Downsampling the tensor to match the dimensions of the left stream ysing a Conv layer
    right_stream = Conv2D(filters=4*filters,kernel_size=1, strides=strides)(tensor)
    right_stream = BatchNormalization()(right_stream)
    
    x = Add()([right_stream, x])
    x = ReLU()(x)
    return x
    

In [22]:
def resnet_block(x, filters, reps, strides):
    x = projection_block(x, filters, strides)
    for _ in range(reps-1):
        x = identity_block(x, filters)
    return x


In [26]:
input_layer = Input(shape=(224, 224, 3))
x = conv_batchnorm_relu(input_layer, filters = 64, kernel_size=7, strides=2)
x = MaxPool2D(pool_size=3, strides=2)(x)
x = resnet_block(x, filters=64, reps=3, strides=1)
x = resnet_block(x, filters=128, reps=4, strides=2)
x = resnet_block(x, filters=256, reps=6, strides=2)
x = resnet_block(x, filters=512, reps=3, strides=2)
x = GlobalAvgPool2D()(x)
output = Dense(units=1000, activation='softmax')(x)

In [27]:
from tensorflow.keras import Model
model = Model(inputs=input_layer, outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_131 (Conv2D)             (None, 112, 112, 64) 9472        input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization_131 (BatchN (None, 112, 112, 64) 256         conv2d_131[0][0]                 
__________________________________________________________________________________________________
re_lu_113 (ReLU)                (None, 112, 112, 64) 0           batch_normalization_131[0][0]    
______________________________________________________________________________________________

**Enhancement**: ResNext

Partition the filters, so that "interchannel interaction" only happens by groups of 4

![alt text](data/resnext_module.png "ResNext")